# Create Train/Test Matrices

Now, taking the bulk data from the recidivism dataset, I will construct train/testing matrices in the format of Triage. The full data table is 1.7G on disk in pickle format, so I will subsample the years for quicker modelling time. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
recid_data = pd.read_pickle('final_recid_data.pkl')

In [3]:
recid_data.head()

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,SENTENCE_END,PAROLE_DAYS,AGE_AT_RELEASE,NUMBER_OF_COUNTS,COUNTY_ALAMANCE,COUNTY_BEAUFORT,COUNTY_BRUNSWICK,COUNTY_BUNCOMBE,...,INMATE_GENDER_CODE_MALE,INMATE_GENDER_CODE_nan,INMATE_RACE_CODE_BLACK,INMATE_RACE_CODE_INDIAN,INMATE_RACE_CODE_OTHER,INMATE_RACE_CODE_UNKNOWN,INMATE_RACE_CODE_WHITE,INMATE_RACE_CODE_nan,PREVIOUS_COMMITMENTS,RECIDIVATED
0,4,AA,1984-07-11,0.0,22,2,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,1.0,0
1,6,AA,1973-03-28,0.0,21,1,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,1.0,0
2,6,AB,1975-08-18,0.0,24,27,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,2.0,0
3,8,AA,1990-05-17,0.0,26,1,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,1.0,0
4,8,AB,1994-01-26,0.0,30,1,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,2.0,1


First, we need to find out which years appear to have full data because the data supplied becomes sparser further back. It appears to begin in earnest in 1973.

In [5]:
recid_data['SENTENCE_END_YEAR'] = recid_data.SENTENCE_END.dt.year

In [23]:
recid_data.groupby('SENTENCE_END_YEAR').agg({'RECIDIVATED':["count", 'mean']})[15:45]

RECIDIVATED          
                        count      mean
SENTENCE_END_YEAR                      
1970                        4  0.000000
1971                       11  0.000000
1972                        5  0.000000
1973                     8377  0.233138
1974                     8622  0.227441
1975                     8370  0.224014
1976                    10911  0.207864
1977                    10668  0.198538
1978                     9068  0.207322
1979                     8927  0.232217
1980                     9644  0.254148
1981                    10944  0.271747
1982                    12198  0.270536
1983                    15642  0.274262
1984                    13027  0.282106
1985                    13645  0.284646
1986                    13754  0.288861
1987                    13481  0.301239
1988                    14265  0.323169
1989                    16597  0.349702
1990                    17014  0.356118
1991                    20129  0.348154
1992                    21522  0.312517
1993                    20796  0.270677
1994                    17905  0.247752
1995                    17608  0.253635
1996                    21945  0.280930
1997                    22256  0.302166
1998                    23362  0.307422
1999                    23132  0.316877

### Getting the Dataset in the Proper Format

In [13]:
features = recid_data[recid_data.columns.difference(['INMATE_DOC_NUMBER', 'INMATE_COMMITMENT_PREFIX', 'RECIDIVATED', 
                                                     'SENTENCE_START', 'SENTENCE_END', 'SENTENCE_END_YEAR'])]

In [19]:
proper_format = pd.concat([recid_data[['INMATE_DOC_NUMBER','SENTENCE_END']], features, recid_data['RECIDIVATED']], axis=1)

In [20]:
proper_format.sort_values(by='SENTENCE_END', inplace=True)

In [21]:
proper_format.rename(columns={'INMATE_DOC_NUMBER': 'entity_id', 'SENTENCE_END':"as_of_date"}, inplace=True)

### Breaking into Matrices

I will use the years from 1980 to 1995 as my testing timeframe, with one matrix per year. I will start my training data at 1975.

In [33]:
matrix_keys = range(16)

In [34]:
seed = 12412
trainings = []
testings = []

train_matrix_uuids = []
train_end_times = []
test_matrix_uuids = []
evaluation_start_times = []
evaluation_end_times = []
model_configs = []
num_features = []


# create and save matrices
for i in matrix_keys:
    test_start_date = pd.to_datetime('1980-01-01') + pd.DateOffset(years = i)
    test_end_date = test_start_date + pd.DateOffset(years = 1)
    
    train_start_date = pd.to_datetime('1975-01-01')
    train_end_date = test_start_date - pd.DateOffset(years = 3)
    
    # Start the training set at 1975
    train_df = proper_format[(proper_format.as_of_date >= train_start_date) &
                             (proper_format.as_of_date < train_end_date)].sample(frac=0.4, random_state=seed)
    
    test_df = proper_format[(proper_format.as_of_date >= test_start_date) &
                            (proper_format.as_of_date < test_end_date)].sample(frac=0.4, random_state=seed)
    
    train_uuid = 'train_{}'.format(i)
    test_uuid = 'test_{}'.format(i)
    
    trainings.append(train_uuid)
    testings.append(test_uuid)
    
    train_matrix_uuids.append(train_uuid)
    train_end_times.append(train_end_date)
    test_matrix_uuids.append(test_uuid)
    evaluation_start_times.append(test_start_date)
    evaluation_end_times.append(test_end_date)
    model_configs.append("")
    num_features.append(train_df.shape[1])
    
    train_df.to_csv('../train_matrices/' + train_uuid + '.csv', index=False)
    test_df.to_csv('../test_matrices/'  + test_uuid  + '.csv', index=False)
    
    print('{} - {} \n\t Train Size: {} \n\t Test Size: {} \n\t Recid Rate: {} \n'.format(test_uuid, 
                                                                test_start_date, train_df.shape[0], 
                                                                test_df.shape[0], test_df.RECIDIVATED.mean()))

test_0 - 1980-01-01 00:00:00 
	 Train Size: 7712 
	 Test Size: 3858 
	 Recid Rate: 0.2547952306894764 

test_1 - 1981-01-01 00:00:00 
	 Train Size: 11980 
	 Test Size: 4378 
	 Recid Rate: 0.27569666514390134 

test_2 - 1982-01-01 00:00:00 
	 Train Size: 15607 
	 Test Size: 4879 
	 Recid Rate: 0.2693174830907973 

test_3 - 1983-01-01 00:00:00 
	 Train Size: 19178 
	 Test Size: 6257 
	 Recid Rate: 0.2686591018059773 

test_4 - 1984-01-01 00:00:00 
	 Train Size: 23035 
	 Test Size: 5211 
	 Recid Rate: 0.29053924390711955 

test_5 - 1985-01-01 00:00:00 
	 Train Size: 27413 
	 Test Size: 5458 
	 Recid Rate: 0.29314767314034446 

test_6 - 1986-01-01 00:00:00 
	 Train Size: 32292 
	 Test Size: 5502 
	 Recid Rate: 0.2880770628862232 

test_7 - 1987-01-01 00:00:00 
	 Train Size: 38549 
	 Test Size: 5392 
	 Recid Rate: 0.3035979228486647 

test_8 - 1988-01-01 00:00:00 
	 Train Size: 43760 
	 Test Size: 5706 
	 Recid Rate: 0.31983876621100593 

test_9 - 1989-01-01 00:00:00 
	 Train Size: 49218 
	

In [37]:
# Create raw_paired_matrices info file
paired_matrices_raw = pd.DataFrame(np.column_stack([train_matrix_uuids, train_end_times, 
                                                    test_matrix_uuids, evaluation_start_times,
                                                    evaluation_end_times, model_configs, num_features]),
                                   columns=['train_matrix', 'train_end_time', 'test_matrix',
                                            'evaluation_start_time', 'evaluation_end_time',
                                            'model_config', 'num_features'])

paired_matrices_raw.to_csv('../paired_matrices_raw.csv', index=False, header=False)

In [38]:
# Write trainings and testings file
with open('../trainings.txt', 'w') as f:
    for item in trainings:
        f.write("%s\n" % item)
        
with open('../testings.txt', 'w') as f:
    for item in testings:
        f.write("%s\n" % item)


In [39]:
paired_matrices_raw

,train_matrix,train_end_time,test_matrix,evaluation_start_time,evaluation_end_time,model_config,num_features
0,train_0,1977-01-01 00:00:00,test_0,1980-01-01 00:00:00,1981-01-01 00:00:00,,289
1,train_1,1978-01-01 00:00:00,test_1,1981-01-01 00:00:00,1982-01-01 00:00:00,,289
2,train_2,1979-01-01 00:00:00,test_2,1982-01-01 00:00:00,1983-01-01 00:00:00,,289
3,train_3,1980-01-01 00:00:00,test_3,1983-01-01 00:00:00,1984-01-01 00:00:00,,289
4,train_4,1981-01-01 00:00:00,test_4,1984-01-01 00:00:00,1985-01-01 00:00:00,,289
5,train_5,1982-01-01 00:00:00,test_5,1985-01-01 00:00:00,1986-01-01 00:00:00,,289
6,train_6,1983-01-01 00:00:00,test_6,1986-01-01 00:00:00,1987-01-01 00:00:00,,289
7,train_7,1984-01-01 00:00:00,test_7,1987-01-01 00:00:00,1988-01-01 00:00:00,,289
8,train_8,1985-01-01 00:00:00,test_8,1988-01-01 00:00:00,1989-01-01 00:00:00,,289
9,train_9,1986-01-01 00:00:00,test_9,1989-01-01 00:00:00,1990-01-01 00:00:00,,289
